### 직방
- 동이름을 입력하면 아파트 매물을 데이터 프레임으로 만들기
- robots.txt

### 1. 웹서비스 분석 : URL 찾기, 크롤링 전략 세우기
- 동이름 입력 -> lat, lng(위도, 경도 가져오기)
- lat, lng -> geohash : geohash 패키지 설치
- geohash -> item ids 가져오기
- item ids -> item datas 가져오기 -> 데이터 프레임으로 만들기

In [3]:
import requests

### 1) 동이름 -> 위도 경도 가져오기

In [6]:
addr = "성수동" # 주소
url = "https://apis.zigbang.com/search?q={}".format(addr)
response = requests.get(url)
data = response.json()["items"][0] # 파싱 # 간단하게 하나의 데이터만 가져오기
lat, lng = data["lat"], data["lng"] # 위도, 경도 데이터

### 2) 위도, 경도 -> geohash 데이터 가져오기

In [8]:
# geohash : 위도, 경도 값을 문자로 바꿔주는 패키지 - > pip install geohash2

In [9]:
import geohash2

In [10]:
# precision : 작을수록 큰범위, 클수록 작은범위
geohash = geohash2.encode(lat, lng, precision = 5)
lat, lng, geohash

(37.541725158691406, 127.04335021972656, 'wydmd')

### 3) geohash -> ids 데이터 가져오기

In [16]:
url = "https://apis.zigbang.com/property/apartments/items?domain=zigbang&geohash={}&q=type=sales%7Cprice=0~-1%7CfloorArea=0~-1".format(geohash)
response = requests.get(url)
datas = response.json()["items"]
ids = [data["itemId"] for data in datas] # 원하는 itemId를 리스트로 만들기
len(data), ids[:3]

(2345, [17788871, 17813458, 17962259])

### 4) ids -> items 데이터 가져오기

In [19]:
ids_str = str(ids[:200]).replace(" ", "") # 데이터의 공백을 없애준다. # 공백이 있으면 에러가 난다.
# 200개의 데이터만 가져온다 2345개를 다 가져오면 너무 많은 데이터로 오류가 난다.

In [22]:
url = "https://apis.zigbang.com/property/apartments/items?\
vritemIds={ids}&itemIds={ids}&citemIds={ids}".format(ids = ids_str)
response = requests.get(url)
datas = response.json()["items"]

In [39]:
# 데이터 프레임으로 만들기
item_df = pd.DataFrame(datas)

In [40]:
len(item_df.columns)

39

In [41]:
# pandas option 설정
# 중간에 columns이 끊어져서 보이는것을 해결
pd.options.display.max_columns # 20개 출력
pd.options.display.max_columns = len(item_df.columns) # 39개

In [42]:
item_df.tail(2)

,agentId,agentName,agentNo,agentPhone,agentTel,agentUserName,agreementArea,agreementAreaName,apartmentId,apartmentName,buildingFloor,buildingName,created_at,exclusive_type,floorPlan,grossArea,grossAreaName,groupedItemFloor,house_key,isNew,itemId,itemImage,itemImages,itemTitle,lat,lng,netArea,real_type,regDate,regTime,rent,roomTypeId,sales,sectionType,type,userNo,viewType,vr_request_id,vr_status
198,2724034,우리공인중개사,NaN,01043873588,0222991900,최성숙,"{'m2': 0, 'p': 0}","{'m2': 144, 'p': 43}",1349,행당한진타운,19,121동,2019-10-05 18:44:14,1,https://ic.zigbang.com/ic/floorplan/1349/3668/...,"{'m2': 144, 'p': 43}","{'m2': 144, 'p': 43}",고층,None,True,18318214,[https://ic.zigbang.com/ic/floorplan/1349/3668...,None,"입주가능, 욕실포함올수리되어 깨끗, 전망좋고 채광좋은집",37.555933,127.028746,"{'m2': 114, 'p': 34}",아파트,2019-11-08,2019-11-08T02:46:43.000Z,0,3668,115000,premium_recommend,매매,2724034,1,None,None
199,2724034,우리공인중개사,NaN,01043873588,0222991900,최성숙,"{'m2': 0, 'p': 0}","{'m2': 104, 'p': 31}",5361,행당대림,15,113동,2019-07-25 15:50:37,1,https://ic.zigbang.com/ic/floorplan/5361/8644/...,"{'m2': 104, 'p': 31}","{'m2': 104, 'p': 31}",저층,None,True,17331412,[https://ic.zigbang.com/ic/floorplan/5361/8644...,None,"작은방확장, 올수리, 중문설치, 깨끗한집",37.557901,127.027117,"{'m2': 84, 'p': 25}",아파트,2019-11-08,2019-11-08T02:46:43.000Z,0,8644,87000,premium_recommend,매매,2724034,1,None,None


In [43]:
# 원하는 column을 필터링
columns = [
    "itemId", "buildingFloor", "groupedItemFloor", "grossArea", 
    "lat", "lng", "sales", "itemTitle" 
]

In [44]:
item_df = item_df[columns]
item_df.tail(2)

,itemId,buildingFloor,groupedItemFloor,grossArea,lat,lng,sales,itemTitle
198,18318214,19,고층,"{'m2': 144, 'p': 43}",37.555933,127.028746,115000,"입주가능, 욕실포함올수리되어 깨끗, 전망좋고 채광좋은집"
199,17331412,15,저층,"{'m2': 104, 'p': 31}",37.557901,127.027117,87000,"작은방확장, 올수리, 중문설치, 깨끗한집"


In [45]:
# grossArea 컬럼 분류
item_df["m2"] = item_df["grossArea"].apply(lambda data: data["m2"])
item_df["p2"] = item_df["grossArea"].apply(lambda data: data["p"])
item_df.drop(columns=["grossArea"], inplace = True)

In [46]:
item_df.tail(2)

,itemId,buildingFloor,groupedItemFloor,lat,lng,sales,itemTitle,m2,p2
198,18318214,19,고층,37.555933,127.028746,115000,"입주가능, 욕실포함올수리되어 깨끗, 전망좋고 채광좋은집",144,43
199,17331412,15,저층,37.557901,127.027117,87000,"작은방확장, 올수리, 중문설치, 깨끗한집",104,31


#### 함수로 만들기
- 동이름 -> 아파트 매물 데이터 프레임이 출력

In [59]:
def crawling_apt(addr):
    # 동이름 -> 위도, 경도 가져오기
    url = "https://apis.zigbang.com/search?q={}".format(addr)
    response = requests.get(url)
    data = response.json()["items"][0] # 파싱 # 간단하게 하나의 데이터만 가져오기
    lat, lng = data["lat"], data["lng"] # 위도, 경도 데이터
    
    # 위도, 경도 -> geohash 데이터 가져오기
    geohash = geohash2.encode(lat, lng, precision = 5)
    
    # geohash -> ids 데이터 가져오기
    url = "https://apis.zigbang.com/property/apartments/items?domain=zigbang&geohash={}&q=type=sales%7Cprice=0~-1%7CfloorArea=0~-1".format(geohash)
    response = requests.get(url)
    datas = response.json()["items"]
    ids = [data["itemId"] for data in datas]
    
    # ids -> items 매물정보 : 200개씩
    dfs = []
    for idx in range(0, len(ids), 200):
        start, end = idx, idx + 200
        ids_str = str(ids[start:end]).replace(" ", "")
        url = "https://apis.zigbang.com/property/apartments/items?\
vritemIds={ids}&itemIds={ids}&citemIds={ids}".format(ids = ids_str)
        response = requests.get(url)
        datas = response.json()["items"]
        item_df = pd.DataFrame(datas)
        columns = ["itemId", "buildingFloor", "groupedItemFloor", "grossArea", "lat", "lng", "sales", "itemTitle" ]
        item_df = item_df[columns]
        item_df["m2"] = item_df["grossArea"].apply(lambda data: data["m2"])
        item_df["p2"] = item_df["grossArea"].apply(lambda data: data["p"])
        item_df.drop(columns=["grossArea"], inplace = True)
        dfs.append(item_df)
        
    result_df = pd.concat(dfs)
    result_df.reset_index(drop=True, inplace=True)
    return result_df

In [60]:
result = crawling_apt("성수동")
result.tail(2)

,itemId,buildingFloor,groupedItemFloor,lat,lng,sales,itemTitle,m2,p2
2343,18414690,21,중층,37.568878,127.031898,85000,집상태 깨끗한 물건,107,32
2344,18408870,25,중층,37.570465,127.024673,118000,시세보다 저렴한 가격으로 투자하기 좋아요 현재 전세거주중,107,32


#### 연습

In [61]:
# 도곡동에서 30층 이상되는 아파트에 고층 매물을 찾아서 구매를 할까 고민

In [71]:
items_df = crawling_apt("도곡동")
items_df.tail(2)

,itemId,buildingFloor,groupedItemFloor,lat,lng,sales,itemTitle,m2,p2
677,18464087,30,저층,37.493337,127.026873,215000,전세 안고 매매 가능 매물,111,33
678,18385710,15,None,37.492310,127.035778,170000,**실입주가능 **입주정상 **로얄층,183,55


In [72]:
items_df.dtypes
# buildingFloor    object

itemId                int64
buildingFloor        object
groupedItemFloor     object
lat                 float64
lng                 float64
sales                 int64
itemTitle            object
m2                    int64
p2                    int64
dtype: object

In [73]:
# items_df["buildingFloor"].astype("int") # 에러
items_df = items_df[items_df["buildingFloor"].notnull()] # Na값 제거
items_df.reset_index(drop=True, inplace=True)
items_df.tail(2)

,itemId,buildingFloor,groupedItemFloor,lat,lng,sales,itemTitle,m2,p2
662,18464087,30,저층,37.493337,127.026873,215000,전세 안고 매매 가능 매물,111,33
663,18385710,15,None,37.492310,127.035778,170000,**실입주가능 **입주정상 **로얄층,183,55


In [74]:
items_df["buildingFloor"] = items_df["buildingFloor"].astype("int")

In [75]:
result_df = items_df[
    (items_df["buildingFloor"] >= 30) &
    (items_df["groupedItemFloor"] == "고층")
]
len(result_df)

91

In [76]:
np.average(result_df["sales"]) # 평균값 확인

232054.94505494504

### 모듈로 만들기

In [77]:
%%writefile zigbang.py
import pandas as pd
import requests
import geohash2

def crawling_apt(addr):
    # 동이름 -> 위도, 경도 가져오기
    url = "https://apis.zigbang.com/search?q={}".format(addr)
    response = requests.get(url)
    data = response.json()["items"][0] # 파싱 # 간단하게 하나의 데이터만 가져오기
    lat, lng = data["lat"], data["lng"] # 위도, 경도 데이터
    
    # 위도, 경도 -> geohash 데이터 가져오기
    geohash = geohash2.encode(lat, lng, precision = 5)
    
    # geohash -> ids 데이터 가져오기
    url = "https://apis.zigbang.com/property/apartments/items?domain=zigbang&geohash={}&q=type=sales%7Cprice=0~-1%7CfloorArea=0~-1".format(geohash)
    response = requests.get(url)
    datas = response.json()["items"]
    ids = [data["itemId"] for data in datas]
    
    # ids -> items 매물정보 : 200개씩
    dfs = []
    for idx in range(0, len(ids), 200):
        start, end = idx, idx + 200
        ids_str = str(ids[start:end]).replace(" ", "")
        url = "https://apis.zigbang.com/property/apartments/items?\
vritemIds={ids}&itemIds={ids}&citemIds={ids}".format(ids = ids_str)
        response = requests.get(url)
        datas = response.json()["items"]
        item_df = pd.DataFrame(datas)
        columns = ["itemId", "buildingFloor", "groupedItemFloor", "grossArea", "lat", "lng", "sales", "itemTitle" ]
        item_df = item_df[columns]
        item_df["m2"] = item_df["grossArea"].apply(lambda data: data["m2"])
        item_df["p2"] = item_df["grossArea"].apply(lambda data: data["p"])
        item_df.drop(columns=["grossArea"], inplace = True)
        dfs.append(item_df)
        
    result_df = pd.concat(dfs)
    result_df.reset_index(drop=True, inplace=True)
    return result_df

Writing zigbang.py
